In [1]:
using Pkg
Pkg.activate("../STLCG.jl/.")

 Activating environment at `~/projects/AutonomousRiskFramework/STLCG.jl/Project.toml`


In [2]:
using Revise
using STLCG
using EllipsisNotation


┌ Info: Precompiling STLCG [6ababdf7-4701-4f28-b3b2-5630f64e3c98]
└ @ Base loading.jl:1278
  ** incremental compilation may be fatally broken for this module **



In [3]:
bs = 2
t = 15
x_dim = 3
dim = 2
x = randn(bs, t, x_dim);
y = randn(bs, t, x_dim);


# rnn_cell(op, xs[1], op.h0; scale, distributed)
# STLCG.gradient(x -> sum(STLCG.rnn_cell(op, x, h0; scale, distributed)[1]), xs[1]) # <---- doesn't work
# STLCG.gradient(x -> sum(STLCG.rnn_cell(op, x, h0; scale, distributed)[2][1][2]), xs[1]) # <--- works

2×15×3 Array{Float64,3}:
[:, :, 1] =
  0.419488  -0.129578  -3.05877   -0.962906  …   1.25893  0.714361  1.51065
 -1.37093    0.61214   -0.361995   0.101949     -1.95622  0.891491  0.254229

[:, :, 2] =
 0.143127   0.522536  1.0208    -1.29722  …  -0.411764  -1.61588   1.72197
 0.109245  -0.437375  0.347346   1.27957      0.797174  -0.226718  0.347156

[:, :, 3] =
 -2.01827  0.579982  0.89579   -0.492036  …  -0.182948  -0.0350106   0.450487
 -1.27709  0.473898  0.301856   1.20094       0.840278  -0.81383    -2.08641

In [4]:
op = Negation(LessThan(:x, 4.0))
op1 = LessThan(:x, 0.0)
op2 = LessThan(:y, 0.0)
or = Or(subformula1=op1, subformula2=op2)
op = Always(subformula=or, interval=nothing)

Always
  subformula: Or
  interval: Nothing nothing
  _interval: Array{Float64}((2,)) [0.0, Inf]
  rnn_dim: Int8 1
  steps: Int8 1
  operation: Minish (function of type typeof(Minish))
  M: Array{Float64}((1, 1)) [0.0]
  b: Array{Float64}((1,)) [1.0]


In [5]:
pscale=1
scale=0
keepdims=true
distributed=true

true

In [6]:
ϕ = op1
ψ = op2

LessThan(:y, 0.0)

In [8]:
Until(ϕ, ψ)((x,y))

2×15×3 Array{Float64,3}:
[:, :, 1] =
 -1.09897    0.129578  -1.30421   …  -0.513777  -0.513777  -0.513777
 -0.522428  -1.45654   -0.801902     -0.207492  -0.207492  -0.207492

[:, :, 2] =
 -0.143127  -1.13627  -1.0208    …  -0.676151    1.13378   -1.16121
 -0.109245  -1.02824  -0.842976     -0.0474823  -0.709525  -0.347156

[:, :, 3] =
 -1.04051  -0.579982  -0.579982  …  -0.364801  -0.393951  -0.393951
 -1.86907  -0.473898  -0.301856     -0.717751  -0.932582   0.638628

In [57]:
LARGE_NUMBER = 1E6
trace1 = ϕ(x)
trace2 = ψ(y)
Alw = Always(subformula=GreaterThan(:z, 0.0), interval=nothing)
LHS = permutedims(repeat(reshape(trace2, (size(trace2)..., 1)), 1,1,1,size(trace2)[2]), [1, 4, 3, 2])
RHS = ones(size(LHS)) * -LARGE_NUMBER
for i in 4:8
    RHS[:,i:end,:,i] = Alw(trace1[:,i:end,:]; pscale, scale, keepdims, distributed)
end
out = Maxish(Minish(cat(LHS, RHS, dims=5); dims=5, scale, keepdims=false, distributed); scale, keepdims=false, distributed, dims=4)

2×15×3 Array{Float64,3}:
[:, :, 1] =
 -1.0e6  -1.0e6  -1.0e6  -1.14929   …  -0.631479  -0.631479  -0.631479
 -1.0e6  -1.0e6  -1.0e6   0.290862     -0.857424  -1.78225   -1.78225

[:, :, 2] =
 -1.0e6  -1.0e6  -1.0e6  -0.861182  …  -1.31951   -1.31951   -1.31951
 -1.0e6  -1.0e6  -1.0e6  -0.342139     -0.833288  -0.833288  -1.01577

[:, :, 3] =
 -1.0e6  -1.0e6  -1.0e6   0.12955  …  -1.6031    -1.6031    -1.6031
 -1.0e6  -1.0e6  -1.0e6  -1.20997     -0.628788  -0.792873  -0.792873

In [58]:
out[1,:,1]

15-element Array{Float64,1}:
 -1.0e6
 -1.0e6
 -1.0e6
 -1.1492900114485687
 -1.1492900114485687
 -1.212407114238258
 -1.212407114238258
 -0.631478841925606
 -0.631478841925606
 -0.631478841925606
 -0.631478841925606
 -0.631478841925606
 -0.631478841925606
 -0.631478841925606
 -0.631478841925606

In [53]:
Maxish(Minish(cat(LHS, RHS, dims=5); dims=5, scale, keepdims=false, distributed); scale, keepdims=false, distributed, dims=4)

2×15×3 Array{Float64,3}:
[:, :, 1] =
 0.110268  -0.570177  -0.570177  …   0.0423217   0.0423217  -0.177004
 0.607287   0.607287  -1.42709      -0.688285   -1.78225    -0.21453

[:, :, 2] =
  0.178606  -0.719736  -0.578093  …  -0.718358  -0.608101  -0.499044
 -0.853146  -0.853146   0.735788      0.538016   1.58135   -1.01577

[:, :, 3] =
  0.206686   0.206686  -1.53311   0.12955  …   0.331363   0.331363  -0.475145
 -0.348084  -0.802872  -1.35152  -1.20997     -0.526954  -0.792873   0.694338

In [96]:
vcat(STLCG.gradient(x -> sum(op((x,x); distributed=true)), x)...)

2×15×3 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  -1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0  -1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  -1.0  0.0  0.0  0.0

In [99]:
vcat(STLCG.gradient((x, y) -> sum(STLCG.robustness_trace(op, (x,y); distributed=true)), x,y)...)[:,:,1]'

15×4 LinearAlgebra.Adjoint{Float64,Array{Float64,2}}:
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  1.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  1.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0

In [9]:
vcat(STLCG.gradient(x -> sum(STLCG.run_rnn_cell(op, x, x, scale=0)[1][:,end,:]), x)...)

2×15×3 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3] =
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [103]:
□ϕ = STLCG.□(op, interval=[2,4])

Always
  subformula: Always
  interval: Array{Int64}((2,)) [2, 4]
  _interval: Array{Int64}((2,)) [2, 4]
  rnn_dim: Int8 4
  steps: Int8 5
  operation: Minish (function of type typeof(Minish))
  M: Array{Float64}((4, 4)) [0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 0.0]
  b: Array{Float64}((4,)) [0.0, 0.0, 0.0, 1.0]


In [141]:
robustness_trace(□ϕ, x; args...)

LoadError: MethodError: no method matching robustness_trace(::Always, ::Array{Float64,3}; keepdims=true, dims=4, distributed=false, scale=0, pscale=1)
Closest candidates are:
  robustness_trace(!Matched::STLCG.Or, ::Any; dims, pscale, scale, keepdims, distributed) at In[54]:44
  robustness_trace(!Matched::STLCG.LessThan, ::Any; pscale, kwargs...) at In[54]:2
  robustness_trace(!Matched::STLCG.Implies, ::Any; pscale, scale, keepdims, distributed) at In[54]:7 got unsupported keyword argument "dims"
  ...

In [ ]:
STLCG.gradient(x -> sum(STLCG.rnn_cell(op, x, h0; scale, distributed)[2][1][2]), xs[1]) # <--- works

In [4]:
ϕ = STLCG.LessThan(:x, 5)
ψ = STLCG.GreaterThan(:x, -3)
op = STLCG.Or(subformula1=STLCG.And(subformula1=ϕ, subformula2=ψ), subformula2=ϕ)

STLCG.Or
  subformula1: STLCG.And
  subformula2: STLCG.LessThan
  operation: Maxish (function of type typeof(Maxish))


In [191]:
(f::STLCG.Formula)(x::Array{Float64}; kwargs...) = robustness_trace(f, x; kwargs...)

In [192]:
ϕ(x)

2×15×3 Array{Float64,3}:
[:, :, 1] =
 4.33878  4.55352  4.0096   4.28465  …  4.87325  4.24155  4.18938  4.36833
 4.35477  4.20984  4.53724  4.8834      4.25997  4.16101  4.62856  4.13871

[:, :, 2] =
 4.19292  4.05989  4.26052  4.3984   …  4.02068  4.74747  4.05824  4.06667
 4.13728  4.53258  4.77843  4.72391     4.18943  4.02867  4.20342  4.66529

[:, :, 3] =
 4.47263  4.89431  4.49562  4.28735  …  4.9606   4.3826   4.59063  4.70413
 4.29403  4.08497  4.01783  4.86847     4.10466  4.29218  4.78882  4.30469

In [147]:
struct Predicate
    x::Symbol
end

In [157]:
(p::Primitive)(x::Int) = (2*x, p.x)
(p::Primitive)(x::Real) = 3*x

In [158]:
a = Primitive(:asadsadsa)

Primitive(:asadsadsa)

In [159]:
a(1), a(1.)

((2, :asadsadsa), 3.0)

In [148]:
Primitive(:x) < 5

Primitive(:x)

In [172]:
Primitive(:a) > Primitive(:b)

false

In [170]:
(Base.:(<))(a::Symbol, b::Primitive) = a.x < b.x

In [75]:
function <(a,b)
    a < b
end

< (generic function with 1 method)

In [78]:
< |> methods

LoadError: syntax: "<" is not a unary operator